In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlparse, parse_qs
import IPython.display as display
import openpyxl
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
import re

def remove_illegal_characters(text):
    """
    Excel에서 허용되지 않는 문자를 제거하는 함수
    """
    if not isinstance(text, str):
        return text
    return re.sub(r'[\x00-\x1F\x7F-\x9F]', '', text)  # ASCII 제어 문자 제거

def shorten_text(text, length=15):
    """
    긴 텍스트를 지정된 길이만큼 줄이고 '...'을 추가하는 함수
    """
    return text[:length] + "..." if len(text) > length else text

def fetch_data(url):
    """
    주어진 URL에서 데이터를 가져와 데이터프레임으로 반환하는 함수
    """
    data = []
    start_count = 0

    while True:
        response = requests.get(f"{url}&iStartCount={start_count}")
        soup = BeautifulSoup(response.text, 'html.parser')
        items = soup.select('.srchResultListW > ul > li')
        
        if not items:
            break
        
        for index, item in enumerate(items):
            # 논문 제목 및 링크 추출
            title_tag = item.select_one('.title a')
            title = remove_illegal_characters(title_tag.text.strip()) if title_tag else '제목 없음'
            link = "https://www.riss.kr" + title_tag['href'] if title_tag and title_tag.has_attr('href') else ''
            excel_link = f'=HYPERLINK("{link}", "논문 보기")' if link else ''

            # 저자, 학회명, 연도 정보 추출
            spans = item.select('.etc > span')
            span_texts = [remove_illegal_characters(span.text.strip()) for span in spans]
            while len(span_texts) < 3:
                span_texts.append('')
            
            # 논문 요약(초록) 15자 제한
            abstract_tag = item.select_one('.cont')
            abstract = remove_illegal_characters(abstract_tag.text.strip()) if abstract_tag else "요약 없음"
            abstract = shorten_text(abstract, 15)  # 15자 제한

            # 데이터 저장
            data.append([start_count + index + 1, title] + span_texts[:3] + [abstract, excel_link])
        
        start_count += 10
    
    columns = ["번호", "제목", "저자", "학회", "연도", "논문 요약", "논문 링크"]
    df = pd.DataFrame(data, columns=columns)
    
    return df

def save_to_excel(df, filename="riss_papers_result.xlsx"):
    """
    데이터프레임을 엑셀 파일로 저장하며 표 형식을 적용하는 함수
    """
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "논문 목록"

    # 테두리 스타일 설정
    thin_border = Border(left=Side(style='thin'), right=Side(style='thin'),
                         top=Side(style='thin'), bottom=Side(style='thin'))
    thick_border = Border(left=Side(style='thick'), right=Side(style='thick'),
                          top=Side(style='thick'), bottom=Side(style='thick'))

    # 헤더 스타일 적용 (배경색 + 굵은 글씨 + 중앙 정렬)
    header_font = Font(bold=True)
    header_fill = PatternFill(start_color="D9D9D9", end_color="D9D9D9", fill_type="solid")  # 연한 회색 배경
    header_alignment = Alignment(horizontal="center", vertical="center")

    ws.append(df.columns.tolist())  # 컬럼명 추가
    for col_idx, col in enumerate(df.columns, 1):
        cell = ws.cell(row=1, column=col_idx)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = header_alignment
        cell.border = thick_border  # 헤더는 굵은 외곽 테두리
        ws.column_dimensions[openpyxl.utils.get_column_letter(col_idx)].width = 20  # 기본 열 너비

    # 데이터 추가 (내부 테두리는 얇은 선)
    for row_idx, row in enumerate(df.itertuples(index=False), 2):
        for col_idx, value in enumerate(row, 1):
            cell = ws.cell(row=row_idx, column=col_idx, value=value)
            cell.alignment = Alignment(horizontal="left", vertical="center")
            cell.border = thin_border

    # 자동 열 너비 조정
    column_widths = {
        "번호": 5, "제목": 30, "저자": 15, "학회": 20, "연도": 10, "논문 요약": 15, "논문 링크": 12
    }
    for col_idx, col in enumerate(df.columns, 1):
        ws.column_dimensions[openpyxl.utils.get_column_letter(col_idx)].width = column_widths.get(col, 20)

    # 엑셀 파일 저장
    wb.save(filename)
    print(f"Data has been saved to {filename}")

def modify_i_start_count(url):
    """
    URL에서 'iStartCount=0'을 제거하는 함수
    """
    return url.replace("iStartCount=0", "") if "iStartCount=0" in url else url

def validate_and_modify_url(url):
    """
    RISS 검색 URL을 검증하고 형식을 수정하는 함수
    """
    search_path = '/search/Search.do?'
    if search_path in url:
        return 'https://www.riss.kr/search/Search.do?' + url.split(search_path, 1)[1]
    return None

# 실행 코드
url = input("Enter RISS URL: ")

validated_url = validate_and_modify_url(url)

if validated_url:
    validated_url = modify_i_start_count(validated_url)
    print(f"Validated URL: {validated_url}")
    df = fetch_data(validated_url)
    display.display(df)
    save_to_excel(df)
else:
    print("Invalid URL. Please enter a valid RISS search URL.")

Enter RISS URL:  https://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&query=LLM&queryText=&iStartCount=0&iGroupView=5&icate=all&colName=re_a_kor&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&pageScale=10&orderBy=&fsearchMethod=search&isFDetailSearch=N&sflag=1&searchQuery=LLM&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&resultKeyword=LLM&pageNumber=1&p_year1=&p_year2=&dorg_storage=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&language_code=&ccl_code=&language=&inside_outside=&fric_yn=&db_type=&image_yn=&regnm=&gubun=&kdc=&ttsUseYn=


Validated URL: https://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&query=LLM&queryText=&&iGroupView=5&icate=all&colName=re_a_kor&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&pageScale=10&orderBy=&fsearchMethod=search&isFDetailSearch=N&sflag=1&searchQuery=LLM&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&resultKeyword=LLM&pageNumber=1&p_year1=&p_year2=&dorg_storage=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&language_code=&ccl_code=&language=&inside_outside=&fric_yn=&db_type=&image_yn=&regnm=&gubun=&kdc=&ttsUseYn=


,번호,제목,저자,학회,연도,논문 요약,논문 링크
0,1,LLM 기반 생성 AI를 활용한 임상시험,"정현철,신건수,김호동,박성빈",한국컴퓨터정보학회,2024,LLM 기반 생성 AI를 활...,"=HYPERLINK(""https://www.riss.kr/search/detail/..."
1,2,도메인 특화 LLM: Mistral 7B를 활용한 금융 업무분야 파인튜닝 및 활용 방법,정천수,한국지능정보시스템학회,2024,도메인 특화 LLM: Mis...,"=HYPERLINK(""https://www.riss.kr/search/detail/..."
2,3,BIM 기반 LLM 모델 개발을 위한 학습 데이터 처리와 파인튜닝 프로세스 개발,강태욱,한국산학기술학회,2024,BIM 기반 LLM 모델 개...,"=HYPERLINK(""https://www.riss.kr/search/detail/..."
3,4,LLM 활용 여부와 진행 단계에 따른 디자인 시나리오 제작에 관한 탐색적 연구,"이다윤,연명흠",디자인융복합학회(구.한국인포디자인학회),2024,LLM 활용 여부와 진행 단...,"=HYPERLINK(""https://www.riss.kr/search/detail/..."
4,5,LLM 모델을 기반으로 한 개인화 NPC의 커뮤니케이션 확장성 고찰,"박선영,김효용",한국애니메이션학회,2023,LLM 모델을 기반으로 한 ...,"=HYPERLINK(""https://www.riss.kr/search/detail/..."
...,...,...,...,...,...,...,...
649,650,Nd:LiLa(MoO4)2 單結晶의 光學的 特性硏究,"배인국,채수천,장영남",한국자원공학회,2004,Nd:LiLa(MoO4)2 ...,"=HYPERLINK(""https://www.riss.kr/search/detail/..."
650,651,"의사소통프로그램이 취업 준비생의 직업기초능력으로서의 의사소통능력, 인지ㆍ정서적 공감...","박홍석,고혜신",한국취업진로학회,2018,의사소통프로그램이 취업 준비...,"=HYPERLINK(""https://www.riss.kr/search/detail/..."
651,652,2-유체 직접 분사기 개발과 가솔린 직분 엔진에의 적용,"이용표(y. p. Lee),장창수(c. s. Jang),김승수(S. S. Kim)",한국자동차공학회,1996,2-유체 직접 분사기 개발과...,"=HYPERLINK(""https://www.riss.kr/search/detail/..."
652,653,『카리타스』로 본 아우구스티누스,문시영 ( Si Young Moon ),한국기독교사회윤리학회,2002,『카리타스』로 본 아우구스티...,"=HYPERLINK(""https://www.riss.kr/search/detail/..."


Data has been saved to riss_papers_result.xlsx
